In [10]:
import datetime
import os
import pdb
from pathlib import Path
from typing import Any

from netCDF4 import Dataset
import numpy as np
import pandas as pd
import torch
import xarray as xr
from earth2studio.data import GEFS_FX, GFS, DataSource
from earth2studio.data.rx import LandSeaMask, SurfaceGeoPotential
from earth2studio.io import IOBackend, NetCDF4Backend
from earth2studio.io.zarr import ZarrBackend
from earth2studio.models.px import GraphCastOperational, GraphCastSmall
from earth2studio.run import deterministic
from earth2studio.utils.type import CoordSystem

SCRATCH = Path(os.getenv("SCRATCH"))

In [3]:
class MemoryDataSource(DataSource):
    def __init__(self, data: xr.DataArray):
        super().__init__()
        self.data = data
    def __call__(self, init_time, variable, **kwargs):
        return self.data


class GFSFill(GFS):
    """
    Intercepts requests for specified variables and provides predefined data arrays.
    For all other variables, it falls back to the standard GFS implementation.
    """

    def __init__(self, custom_arrays: dict[str, xr.DataArray], *args, **kwargs):
        """
        Initializes with a dictionary of custom xarray.DataArrays,
        where the keys are the variable names.
        """
        super().__init__(*args, **kwargs)
        self.custom_arrays = custom_arrays
        self.custom_vars = list(custom_arrays.keys())

    def __call__(
        self,
        time: np.ndarray,
        variable: np.ndarray,
    ) -> xr.DataArray:
        # Identify which custom variables are in the current request
        requested_custom_vars = [v for v in self.custom_vars if v in variable]

        # Identify which variables need to be fetched from the standard GFS source
        other_vars = [v for v in variable if v not in self.custom_vars]

        # Fetch data for non-custom variables from the parent GFS class
        if other_vars:
            gfs_data = super().__call__(time, np.array(other_vars))
        else:
            gfs_data = None

        # Prepare a list of all data arrays to be concatenated
        data_to_concat = []
        if gfs_data is not None:
            data_to_concat.append(gfs_data)

        # Add the requested custom arrays to the list
        for var_name in requested_custom_vars:
            # Expand dims to match the structure of the fetched data
            custom_da = self.custom_arrays[var_name].expand_dims({"time": time})
            data_to_concat.append(custom_da)

        # Concatenate all data arrays along the 'variable' dimension
        if len(data_to_concat) > 1:
            return xr.concat(data_to_concat, dim="variable")
        elif len(data_to_concat) == 1:
            return data_to_concat[0]
        else:
            # Should not happen if 'variable' is never empty
            return xr.DataArray()

In [20]:
# Define a custom IO class that subsets the data before writing to NetCDF.
class SubsetNetCDF4Backend(IOBackend):
    def __init__(
        self, file_name: str, lat_slice: slice, lon_slice: slice, backend_kwargs: dict = {}
    ):
        self.file_name = file_name
        self.lat_slice = lat_slice
        self.lon_slice = lon_slice
        self.backend_kwargs = backend_kwargs
        self.writer = None

    def add_array(
        self, coords: CoordSystem, array_name: str | list[str], **kwargs: dict[str, Any]
    ) -> None:
        # Create a temporary xarray object to correctly select coordinate values.
        # We don't need real data, just the coordinates and dimensions.
        dummy_data = np.zeros([len(v) for v in coords.values()])
        temp_da = xr.DataArray(dummy_data, coords=coords, dims=list(coords.keys()))

        # Select the subset using coordinate values (degrees)
        subset_da = temp_da.sel(lat=self.lat_slice, lon=self.lon_slice)

        # Extract the subsetted coordinates as a dictionary
        subset_coords = {k: v.values for k, v in subset_da.coords.items()}

        # Initialize the internal NetCDF4Backend with the subsetted coordinates
        self.writer = NetCDF4Backend(self.file_name, self.backend_kwargs)
        self.writer.add_array(subset_coords, array_name, **kwargs)

    def write(
        self,
        x: torch.Tensor | list[torch.Tensor],
        coords: CoordSystem,
        array_name: str | list[str],
    ) -> None:
        if self.writer is None:
            raise RuntimeError("add_array must be called before write.")

        if not isinstance(x, list):
            x = [x]
            array_name = [array_name]

        for i, tensor in enumerate(x):
            var_name = array_name[i]

            # 1. Create a DataArray from the incoming global data tensor
            temp_da = xr.DataArray(tensor.cpu().numpy(), coords=coords, dims=list(coords.keys()))

            # 2. Select the desired subset using coordinate values (degrees)
            subset_da = temp_da.sel(lat=self.lat_slice, lon=self.lon_slice)

            # 3. Extract the subsetted data and coordinates for the writer
            subset_x = torch.from_numpy(subset_da.data)
            subset_coords = {k: v.values for k, v in subset_da.coords.items()}

            # 4. Write the subsetted data using the internal writer
            self.writer.write(subset_x, subset_coords, var_name)

    def close(self):
        if self.writer:
            self.writer.close()


def run(init_time, model, members=["gec00"] + [f"gep{p:02d}" for p in range(1, 31)]):
    # The initialization time for the forecast
    # GEFS data is available at 00, 06, 12, 18 UTC.
    forecast_length = 240
    forecast_step_hours = 6
    nsteps = forecast_length // forecast_step_hours

    # Define the output directory for the ensemble forecasts
    output_dir = f"/glade/derecho/scratch/ahijevyc/ai-models/output/graphcast/{init_time:%Y%m%d%H}"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Ensemble forecast outputs will be saved in: {output_dir}")
    print("Initializing model...")
    model = model.load_model(model.load_default_package())
    # Get the grid the model expects by constructing it from its input coordinates
    lat = model.input_coords()["lat"]
    lon = model.input_coords()["lon"]

    z = SurfaceGeoPotential(cache=False)([init_time]).sel(lat=lat, lon=lon)
    z["variable"] = ["z"]  # rename zsl->z
    lsm = LandSeaMask(cache=False)([init_time]).sel(lat=lat, lon=lon)
    model_variables = model.input_coords()["variable"]
    vars_to_zero_fill = [v for v in model_variables if v.startswith("w") or v == "tp06"]

    vars_to_fetch = [v for v in model_variables if v not in vars_to_zero_fill]
    vars_to_fetch.remove("z")
    vars_to_fetch.remove("lsm")

    # 4. The Main Ensemble Loop
    for member in members:
        # Define the output path and the lat/lon slices
        output_filepath = os.path.join(output_dir, f"{member}.nc")
        lat_slice = slice(20, 60)
        lon_slice = slice(220, 300)

        # Check if a valid and complete output file already exists.
        if os.path.exists(output_filepath):
            try:
                # Use xarray to open the dataset. This is a more robust check for
                # a valid, closed NetCDF file than just using the netCDF4 library.
                with xr.open_dataset(output_filepath) as ds:
                    if len(ds.data_vars) != 85:
                        raise ValueError(f"Incorrect # of data vars. Expected 85, found {len(ds.data_vars)}.")
                    for dim_name, dim_size in ds.dims.items():
                        if dim_size == 0:
                            raise ValueError("Dim '{dim_name}' has size 0.")
                    print(f"Valid and complete forecast file already exists for member '{member}', skipping.")
                    continue  # Skip to the next member
            except Exception as e:
                # This will catch errors if the file is corrupt, not a valid NetCDF,
                # or failed our completeness check.
                print(f"Found invalid or incomplete file for member '{member}', removing. Error: {e}")
                os.remove(output_filepath)

        # 4a. Fetch initial conditions for the current member
        print(f"Fetching initial conditions for {member} at {init_time.isoformat()}...")
        gefs_source = GEFS_FX(member=member)
        initial_state_partial = gefs_source(init_time, [datetime.timedelta(hours=0)], vars_to_fetch)

        # Manually regrid the data to the target grid
        print(f"Regridding initial state for {member}...")
        wrapped = initial_state_partial.sel(lon=0).assign_coords(lon=360)
        initial_state_periodic = xr.concat([initial_state_partial, wrapped], dim="lon")
        initial_state_partial = initial_state_periodic.interp(lat=lat, lon=lon, method="linear")

        data_arrays_to_concat = [initial_state_partial]
        for var_name in vars_to_zero_fill:
            zero_array = xr.zeros_like(initial_state_partial.isel(variable=0))
            zero_array["variable"] = var_name
            data_arrays_to_concat.append(zero_array)
        data_arrays_to_concat.extend([z, lsm])
        initial_state = xr.concat(data_arrays_to_concat, dim="variable", coords="minimal")
        assert initial_state.notnull().all()
        # The 'lead_time' dimension must be removed for the MemoryDataSource
        initial_state = initial_state.sel(variable=model_variables).squeeze(
            dim="lead_time", drop=True
        )

        # 4b. Run the forecast using the fetched initial state
        in_memory_source = MemoryDataSource(initial_state)

        # Instantiate our custom writer
        subset_writer = SubsetNetCDF4Backend(
            file_name=output_filepath,
            lat_slice=lat_slice,
            lon_slice=lon_slice,
            backend_kwargs={"mode": "w"},
        )

        # Run the deterministic forecast
        deterministic([init_time], nsteps, model, in_memory_source, subset_writer)

        # Close the writer to finalize the file on disk
        subset_writer.close()

        print(f"Successfully created subset forecast file: {output_filepath}")
        print(f"--- Finished forecast for member '{member}' ---")

    print("\n✅ All ensemble member forecasts have been successfully generated.")


# Use Python's standard datetime object instead of pandas.Timestamp
init_time = datetime.datetime(2023, 4, 25)
run(init_time, GraphCastOperational)

Ensemble forecast outputs will be saved in: /glade/derecho/scratch/ahijevyc/ai-models/output/graphcast/2023042500
Initializing model...
Found invalid or incomplete file for member 'gec00', removing. Error: Incorrect # of data vars. Expected 85, found 0.
Fetching initial conditions for gec00 at 2023-04-25T00:00:00...


Fetching GEFS data:   0%|          | 0/69 [00:00<?, ?it/s]

2025-08-29 13:30:23.749 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f000 27847690-233661
2025-08-29 13:30:23.767 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f000 2103948-119132
2025-08-29 13:30:23.783 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f000 38808859-234597
2025-08-29 13:30:23.797 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f000 32556289-112826
2025-08-29 13:30:23.812 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2ap5/gec00.t00z.pgrb2a.0p50.f000 5039115-167943
2025-08-29 13:30:23.827 | DEBUG    | 

Fetching GEFS data:   0%|          | 0/69 [00:00<?, ?it/s]

2025-08-29 13:30:23.944 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2bp5/gec00.t00z.pgrb2b.0p50.f000 50370237-211535


...

2025-08-29 13:31:42.271 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2ap5/gep23.t00z.pgrb2a.0p50.f000 1350494-253930
2025-08-29 13:31:42.285 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2ap5/gep23.t00z.pgrb2a.0p50.f000 4930881-221042
2025-08-29 13:31:42.300 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2bp5/gep23.t00z.pgrb2b.0p50.f000 42062299-259908
2025-08-29 13:31:42.314 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2ap5/gep23.t00z.pgrb2a.0p50.f000 3999009-223346
2025-08-29 13:31:42.330 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2ap5/gep23.t00z.pgrb2a.0p50.f000 7892550-261994
2025-08-29 13:31:42.344 | DEBUG    | ea

Fetching GEFS data: 100%|██████████| 69/69 [00:01<00:00, 67.62it/s]


2025-08-29 13:31:42.417 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2bp5/gep23.t00z.pgrb2b.0p50.f000 30558656-123925
2025-08-29 13:31:42.432 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2ap5/gep23.t00z.pgrb2a.0p50.f000 3112883-126672
2025-08-29 13:31:42.446 | DEBUG    | earth2studio.data.gefs:fetch_array:405 - Fetching GEFS grib file: noaa-gefs-pds/gefs.20230425/00/atmos/pgrb2bp5/gep23.t00z.pgrb2b.0p50.f000 40855512-242033
Regridding initial state for gep23...
2025-08-29 13:31:43.606 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-08-29 13:31:43.607 | INFO     | earth2studio.run:deterministic:82 - Inference device: cuda
2025-08-29 13:31:43.919 | SUCCESS  | earth2studio.run:deterministic:106 - Fetched data from MemoryDataSource
2025-08-29 13:31:43.929 | INFO     | earth2studio.run:deterministic:136 - Inferen


Running inference: 100%|██████████| 41/41 [00:33<00:00,  1.22it/s]

2025-08-29 13:32:17.451 | SUCCESS  | earth2studio.run:deterministic:146 - Inference complete
Successfully created subset forecast file: /glade/derecho/scratch/ahijevyc/ai-models/output/graphcast/2023042500/gep23.nc
--- Finished forecast for member 'gep23' ---
Valid and complete forecast file already exists for member 'gep24', skipping.
Valid and complete forecast file already exists for member 'gep25', skipping.
Valid and complete forecast file already exists for member 'gep26', skipping.
Valid and complete forecast file already exists for member 'gep27', skipping.
Valid and complete forecast file already exists for member 'gep28', skipping.
Valid and complete forecast file already exists for member 'gep29', skipping.
Valid and complete forecast file already exists for member 'gep30', skipping.

✅ All ensemble member forecasts have been successfully generated.


In [19]:
xr.open_dataset(f"/glade/derecho/scratch/ahijevyc/ai-models/output/graphcast/{init_time:%Y%m%d%H}/gep23.nc")

<xarray.Dataset> Size: 3kB
Dimensions:    (time: 1, lead_time: 41, lat: 0, lon: 321)
Coordinates:
  * time       (time) datetime64[ns] 8B 2023-04-25
  * lead_time  (lead_time) int64 328B 0 21600 43200 ... 820800 842400 864000
  * lat        (lat) float64 0B 
  * lon        (lon) float64 3kB 220.0 220.2 220.5 220.8 ... 299.5 299.8 300.0
Data variables: (12/85)
    t2m        (time, lead_time, lat, lon) float32 0B ...
    msl        (time, lead_time, lat, lon) float32 0B ...
    u10m       (time, lead_time, lat, lon) float32 0B ...
    v10m       (time, lead_time, lat, lon) float32 0B ...
    tp06       (time, lead_time, lat, lon) float32 0B ...
    t50        (time, lead_time, lat, lon) float32 0B ...
    ...         ...
    q700       (time, lead_time, lat, lon) float32 0B ...
    q850       (time, lead_time, lat, lon) float32 0B ...
    q925       (time, lead_time, lat, lon) float32 0B ...
    q1000      (time, lead_time, lat, lon) float32 0B ...
    z          (time, lead_time, lat, lon) float32 0B ...
    lsm        (time, lead_time, lat, lon) float32 0B ...

In [4]:
model = GraphCastOperational.load_model(GraphCastOperational.load_default_package())
ds = GFS()
# cache=False to avoid AttributeError: type object 'WholeFileCacheFileSystem'
# has no attribute '_cat_file'. Did you mean: 'cat_file'?
# dummy time list for required positional argument 'time'
# squeeze 'time' to avoid ValueError: Dimension time already exists.
zsl = SurfaceGeoPotential(cache=False)([0]).squeeze(dim="time")
lsm = LandSeaMask(cache=False)([0]).squeeze(dim="time")

# --- Instantiate Custom Data Source ---
custom_data = {"zsl": zsl, "lsm": lsm}
ds_fill = GFSFill(custom_arrays=custom_data)

# --- Run Forecast ---
nsteps = 8
ofile = SCRATCH / "GFS.nc"
if os.path.exists(ofile):
    os.remove(ofile)
io = NetCDF4Backend(ofile, backend_kwargs={"mode": "w"})
deterministic([init_time], nsteps, model, ds, io)
ofile = SCRATCH / "GFSFill.nc"
if os.path.exists(ofile):
    os.remove(ofile)
io = NetCDF4Backend(ofile, backend_kwargs={"mode": "w"})
deterministic([init_time], nsteps, model, ds_fill, io)

print("Forecast run complete.")

2025-08-27 14:40:32.003 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-08-27 14:40:32.003 | INFO     | earth2studio.run:deterministic:82 - Inference device: cuda
2025-08-27 14:40:32.046 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id tp06 not found in GFS lexicon, good luck
2025-08-27 14:40:32.047 | WARNING  | earth2studio.data.gfs:_create_tasks:324 - Variable tp06 not found in index file for time 2024-04-30 18:00:00 at 0:00:00, values will be unset
2025-08-27 14:40:32.049 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id w50 not found in GFS lexicon, good luck
2025-08-27 14:40:32.049 | WARNING  | earth2studio.data.gfs:_create_tasks:324 - Variable w50 not found in index file for time 2024-04-30 18:00:00 at 0:00:00, values will be unset
2025-08-27 14:40:32.049 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id w100 not found in GFS lexicon, good luck
2025-08-27 14:40:32.049 | WARNING  | earth2studio.data

Fetching GFS data:   0%|          | 0/70 [00:00<?, ?it/s]

2025-08-27 14:40:32.053 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 201324002-786415
2025-08-27 14:40:32.120 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 159577868-1095948
2025-08-27 14:40:32.188 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 196511814-950661


Fetching GFS data:   0%|          | 0/70 [00:00<?, ?it/s]

2025-08-27 14:40:32.286 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 344150566-1247252
2025-08-27 14:40:32.387 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 248847823-585836
2025-08-27 14:40:32.444 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 285528015-821760


...

2025-08-27 14:41:44.310 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 212899983-1201117


Fetching GFS data: 100%|██████████| 70/70 [00:02<00:00, 28.20it/s]


2025-08-27 14:41:44.362 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 242292524-724164
2025-08-27 14:41:44.799 | SUCCESS  | earth2studio.run:deterministic:106 - Fetched data from GFSFill
2025-08-27 14:41:44.806 | INFO     | earth2studio.run:deterministic:136 - Inference starting!



Running inference: 100%|██████████| 9/9 [00:08<00:00,  1.08it/s]

2025-08-27 14:41:53.116 | SUCCESS  | earth2studio.run:deterministic:146 - Inference complete
Forecast run complete.


In [37]:
ifile = SCRATCH/f"ai-models/output/graphcast/{init_time:%Y%m%d%H}/gep01.nc"
print(ifile)
ds = xr.open_dataset(ifile)
ds.z100.squeeze().max(dim=["lat", "lon"])

/glade/derecho/scratch/ahijevyc/ai-models/output/graphcast/2023042400/gep01.nc


<xarray.DataArray 'z100' (lead_time: 3)> Size: 12B
array([163454.97, 163401.  , 163464.33], dtype=float32)
Coordinates:
    time       datetime64[ns] 8B 2023-04-24
  * lead_time  (lead_time) int64 24B 0 6 12

In [9]:
ds = xr.open_dataset(SCRATCH/f"ai-models/output/graphcast/{init_time:%Y%m%d%H}/gep01.nc")
ds.z100.squeeze().max(dim=["lat","lon"])

<xarray.DataArray 'z100' (lead_time: 3)> Size: 12B
array([163468.14,       nan,       nan], dtype=float32)
Coordinates:
    time       datetime64[ns] 8B 2023-04-24
  * lead_time  (lead_time) int64 24B 0 6 12

In [14]:
ds = xr.open_zarr(SCRATCH/f"ai-models/output/graphcast/{init_time:%Y%m%d%H}/gep23")
ds

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 1, lead_time: 5, lat: 721, lon: 1440)
Coordinates:
  * lon        (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * lead_time  (lead_time) timedelta64[h] 40B 00:00:00 ... 1 days 00:00:00
  * time       (time) datetime64[ns] 8B 2023-04-24
  * lat        (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
Data variables: (12/85)
    u150       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    v850       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    u600       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    z          (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    w100       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    v10m       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    ...         ...
    q700       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    z400       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    q850       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    z850       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    q300       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>
    v925       (time, lead_time, lat, lon) float32 21MB dask.array<chunksize=(1, 5, 721, 1440), meta=np.ndarray>

In [20]:
ds.squeeze().max(dim=["lat","lon"]).load()

<xarray.Dataset> Size: 2kB
Dimensions:    (lead_time: 5)
Coordinates:
    time       datetime64[ns] 8B 2023-04-24
  * lead_time  (lead_time) int64 40B 0 6 12 18 24
Data variables: (12/85)
    t2m        (lead_time) float32 20B 310.2 nan nan nan nan
    msl        (lead_time) float32 20B 1.076e+05 nan nan nan nan
    u10m       (lead_time) float32 20B 25.44 nan nan nan nan
    v10m       (lead_time) float32 20B 24.47 nan nan nan nan
    tp06       (lead_time) float32 20B 0.0 nan nan nan nan
    t50        (lead_time) float32 20B 228.9 nan nan nan nan
    ...         ...
    q700       (lead_time) float32 20B 0.0115 nan nan nan nan
    q850       (lead_time) float32 20B 0.0185 nan nan nan nan
    q925       (lead_time) float32 20B 0.02247 nan nan nan nan
    q1000      (lead_time) float32 20B 0.02671 nan nan nan nan
    z          (lead_time) float32 20B 5.746e+04 0.0 0.0 0.0 0.0
    lsm        (lead_time) float32 20B 1.0 0.0 0.0 0.0 0.0